In [1]:

import sys
import tensorflow as tf

import pandas as pd
import numpy as np
import Config
import os
from os.path import isfile,join
import xml.etree.cElementTree as ET
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import time
#import doc2vec_impl
import gensim


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:

batch_size=10
embedding_filename = "glove_6B_200d.txt"
training_epochs=1;



In [3]:
dirpath = "corpus/fulltext/"
files = [f for f in os.listdir(dirpath) if isfile(join(dirpath, f))]



In [4]:
stop_words_list=set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

##########loading dev2vec=============
#doc2vec_impl.main();




d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec_models/doc2vec.model')
#start testing
#printing the vector of document at index 1 in docLabels
docvec = d2v_model.docvecs[files[0]]
print(docvec)


[-1.9957069  -1.7302711  -1.2416555  -1.25041    -0.8760755   1.4730991
  1.2059473   0.38232905 -1.6187527   1.305874    1.1770602   2.308416
  0.02343252 -0.2502587   1.7815354  -1.332049    0.38943136 -2.052053
 -1.3515872  -1.5760187  -0.59295475 -0.66141963  1.3531227   0.47351202
  0.80553395  2.9402657   2.2971811   0.9491949   4.030078    0.6117527
 -0.27292827 -1.1015353  -0.3637588  -1.0808604  -0.38145977 -2.5174286
  1.1072197   2.530051    2.5857248  -1.1959172   1.7793411   0.09467691
 -1.4371932  -2.9791405   1.0630684   0.9067003   0.43923157  3.4134736
  0.47711533 -1.3631905  -3.3734288  -0.2367886   0.3390342  -0.39412883
  0.5333661   0.7227717   0.55354226  0.35878623  0.932402    2.4869206
 -0.55152404  2.4435148  -0.3364468  -1.9685099  -1.7815394  -0.6749915
  2.4418046  -2.3903039   1.1918342   0.36871168 -1.7374214   0.7597355
  1.6034806  -0.9551855   0.8539238  -1.5502286   0.8968359  -0.62780535
 -1.7566789   4.206336    0.92475533  0.4445385   1.4446664   

In [5]:
#
def read_data(file_name):
    with open(file_name, 'r') as f:
        word_vocab = set()  # not using list to avoid duplicate entry
        word_count = {};
        word_index={};

        word2vector = {}
        count=0;
        for line in f:
            line_ = line.strip()  # Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word_index[words_Vec[0]]=count;
            count=count+1;

            word2vector[words_Vec[0]] = np.array(words_Vec[1:], dtype=float)
    print("Total Words in DataSet:", len(word_vocab))
    return word_vocab, word2vector, word_index

In [6]:
def load_embeddings(filename):
    dvocab, w2v,word_index  = read_data(filename);
    return w2v,word_index

In [7]:

embedding_array,word_index = load_embeddings(embedding_filename)
print("Loaded embeddings")

words_data=pd.read_csv("words_data.csv");
catchphrases_data=pd.read_csv("catch_data.csv")
print("reading data completed");

Total Words in DataSet: 400000
Loaded embeddings
reading data completed


In [8]:
print(word_index['unk'])


201534


In [9]:
def get_statements(file):
    with open(dirpath + file, 'r',encoding="utf-8", errors='replace') as f:
        data=str(f.read());
        data=data.lower()
        data = data.replace("\"id=", "id=\"");
        data=data.replace("\n","")
        data=data.replace('".*?=.*?"', "",)
        data=data.replace("&","");
        xml = ET.fromstring(str(data))
        name=None;
        rows_list=[];
        catchphrases=[];
        sentences=[];
        for child in xml:
            if child.tag=="catchphrases":
                for catchphrase in child:
                    id=catchphrase.attrib.get("id")
                    #print(catchphrase.text)
                    #catchphrases.append({"file_id":file,"Name":name,"Id":id,"text":catchphrase.text})
                    catchphrases+=tokenizer.tokenize(catchphrase.text)
            if child.tag=="sentences":
                for sentence in child:
                    id = sentence.attrib.get("id")
                    #sentences.append({"file_id":file,"Name":name,"Id":id,"text":sentence.text})
                    sentences+=tokenizer.tokenize(sentence.text)
    
    
    sentences = [w for w in sentences if not w in stop_words_list] 
    catchphrases = [w for w in catchphrases if not w in stop_words_list]
    
    
    #sentences=[word.lower() for word in sentences if word.isalphanum()]
    #catchphrases=[word.lower() for word in catchphrases if word.isalphanum()]
    
    return sentences,catchphrases


    

In [59]:
def prepare_index_matrices(sentence_indexes):
    length=len(sentence_indexes);
    
    sentence_indexes=[word_index['unk'],word_index['unk']]+sentence_indexes+[word_index['unk'],word_index['unk']];    
    out_indices=[]
    for i in range(2,length):
        out_indices.append(sentence_indexes[i-2:i+3]);
        
    return out_indices;

def prepare_label_matrices(labels):
    length=len(labels);
    labels=[0,0]+labels+[0,0];
    out_labels=[];
    
    for i in range(2,length):
        out_labels.append(labels[i]);
        
    return out_labels;
    

In [124]:
def lookup_indexes_labels(sentences,catchphrases):
    
    sentence_indexes=[];
    labels=[];
    
    for word in sentences:
        if word in word_index:
            sentence_indexes.append(word_index[word]);
        else:
            #print("unknown word",word)
            sentence_indexes.append(word_index['unk']);
            
    for word in sentences:
        if word in catchphrases:
            labels.append(1.0);
        else:
            #print("unknown word",word)
            labels.append(0.0);
            
    return sentence_indexes,labels;

In [125]:
sentences,catchphrases=get_statements(files[0]);
sentence_indexes,labels=lookup_indexes_labels(sentences,catchphrases)
print(len(sentence_indexes),len(labels))

sentence_index_matrix=prepare_index_matrices(sentence_indexes);
label_matrix=prepare_label_matrices(labels);

print(len(sentence_index_matrix),len(label_matrix))

2966 2966
2964 2964


In [274]:

class CatchPhraseExtractor(object):

    def __init__(self, graph, embedding_array):
        self.total_loss=0.0
        self.build_graph(graph, embedding_array)

    def build_graph(self, graph, embedding_array):
        print("generating graph");

        with graph.as_default():

            #self.embeddings = tf.Variable(embedding_array, dtype=tf.float32);
            W = tf.Variable(tf.constant(0.0, shape=[400000, 200 ]),trainable=True, name="W")
            self.embedding_placeholder = tf.placeholder(tf.float32, [400000, 200])
            self.embedding_init = W.assign(self.embedding_placeholder)
            
            
            
            self.train_inputs = tf.placeholder(tf.int32, shape=[None,5])
            self.train_labels = tf.placeholder(tf.int32, shape=[None,1])
            
            sentence_vectors=tf.nn.embedding_lookup(W, self.train_inputs)
            sentence_vectors = tf.reshape(sentence_vectors, [tf.shape(sentence_vectors)[0], 1000])
    
            
            conv_filter=tf.Variable(tf.truncated_normal((1000,200), stddev=0.1));
            word_features=tf.nn.relu(tf.matmul(sentence_vectors, conv_filter));
            
            doc_vector=tf.reduce_max(word_features,reduction_indices=[0],keepdims=True);
            print(doc_vector)
            
            sentence_doc_dummy=tf.tile(doc_vector, [tf.shape(word_features)[0], 1])
            sentence_tensor = tf.reshape(tf.concat([word_features, sentence_doc_dummy],1),[tf.shape(word_features)[0],400])

            weights_input=tf.Variable(tf.random_normal((400,100), stddev=0.2));
            weights_input_2=tf.Variable(tf.random_normal((100,1), stddev=0.2));
            
            #weights_input=tf.Variable(tf.ones((400,200)));
            #weights_input_2=tf.Variable(tf.ones((200,1)));
            
            bias1=tf.Variable(tf.random_normal((100,1), stddev=0.2));
            bias2=tf.Variable(tf.random_normal((1,1), stddev=0.2));
            
            self.sentence_values=self.MLP(sentence_tensor,weights_input,weights_input_2,bias1,bias2);
            
            prediction_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.train_labels,logits=self.sentence_values)
            
            l2_input = tf.nn.l2_loss(weights_input)
            l2_output = tf.nn.l2_loss(weights_input_2)

            regularized_loss=l2_input + l2_output
            regularized_loss=1e-1 * regularized_loss

            #calculating the total loss:sum of prediction loss and regularized loss
            total_loss = prediction_loss + regularized_loss
            
            self.loss=tf.reduce_mean(total_loss)
            
            #using the gradient descent optimizer
            optimizer = tf.train.GradientDescentOptimizer(0.1)
            grads = optimizer.compute_gradients(self.loss)
            #clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
            self.app = optimizer.apply_gradients(grads)

            self.init = tf.global_variables_initializer()
            
    def MLP(self,x,weights_input,weights_input_2,bias1,bias2):
        h1=(tf.matmul( weights_input,x,transpose_a=True,transpose_b=True));
        h1=h1+bias1;
        
        h1=tf.nn.tanh(h1);
        
        h2=tf.matmul(weights_input_2,h1,transpose_a=True,transpose_b=False);
        h2=h2+bias2;
        h2=tf.nn.sigmoid(h2);
        return tf.transpose(h2);
    
    
    def gradient_descent(self,total_loss):
        total_loss=tf.convert_to_tensor(total_loss)
        with tf.GradientTape(persistent=True) as tape:
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            print(variables)
        return total_loss, tape.gradient(tf.convert_to_tensor(total_loss), variables)
        
    
    def train(self, sess, num_steps):
        
        self.init.run()
        print("Initailized")
        total_loss=0.0;
        for file in files[:1]:
            sentences,catchphrases=get_statements(file);
            
            
            sentence_indexes,labels=lookup_indexes_labels(sentences,catchphrases)
            
            sentence_index_matrix=prepare_index_matrices(sentence_indexes);
            label_matrix=prepare_label_matrices(labels);
            
            sentence_index_matrix=np.asarray(sentence_index_matrix)
            label_matrix=np.asarray(label_matrix)
        
            #sentence_indexes=np.reshape(sentence_indexes,(sentence_indexes.shape[0],1))
            label_matrix=np.reshape(label_matrix,(label_matrix.shape[0],1))
            
            
            feed_dict = {self.train_inputs: sentence_index_matrix, self.train_labels: label_matrix}
            #loss_val = sess.run([self.loss], feed_dict=feed_dict)  
            _,loss_val,outputs = sess.run([self.app,self.loss,self.sentence_values], feed_dict=feed_dict)

            for i in range(len(outputs)):
                if label_matrix[i]==1.0:
                    print(outputs[i],label_matrix[i])
            print("Train Finished.",loss_val)
        
    def load_embeddings(self,embedding_array):
        sess.run(self.embedding_init, feed_dict={self.embedding_placeholder: np.asarray(list(embedding_array.values()))})
        

    

In [275]:
def get_features(inputs):
    out=np.zeros((len(inputs),200));
    for i in range(len(inputs)):
        try:
            c = w2v[inputs[i]]
        except KeyError:
            c = np.zeros((1,200));
        out[i]=c;
    
    return np.transpose(out);

In [276]:
graph = tf.Graph()

model = CatchPhraseExtractor(graph, embedding_array)


generating graph
Tensor("Max:0", shape=(1, 200), dtype=float32)


In [277]:
with tf.Session(graph=graph) as sess:
  
    model.load_embeddings(embedding_array)    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    global_step = tf.train.get_or_create_global_step()
    for i in range(1):
        model.train(sess, 10)   
    #optimizer.apply_gradients(zip(grads, model.variables), global_step)
        

Initailized
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [